<a href="https://colab.research.google.com/github/RCCyber/RCCyber/blob/main/detect_employees.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, Model
from keras.applications.inception_v3 import InceptionV3
from keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.optimizers import SGD
from keras.layers import Dense, Activation, Flatten, Dropout

from keras import backend as K
K.image_dim_ordering='th'

import numpy as np
import pandas as pd
import h5py

import matplotlib.pyplot as plt

In [ ]:
!cd /content/sample_data/ && unzip data.zip

In [ ]:
inc_model=InceptionV3(include_top=False, 
                      weights='imagenet', 
                      input_shape=((116, 116, 3)))

In [ ]:
bottleneck_datagen = ImageDataGenerator(rescale=1./255)  #собственно, генератор
    
train_generator = bottleneck_datagen.flow_from_directory('/content/sample_data/data/train/',
                                        target_size=(116, 116),
                                        batch_size=32,
                                        class_mode=None,
                                        shuffle=False)

validation_generator = bottleneck_datagen.flow_from_directory('/content/sample_data/data/validation/', 
                                                              target_size=(116, 116),
                                                               batch_size=32,
                                                               class_mode=None,
                                                               shuffle=False)

Found 2000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.


In [ ]:
bottleneck_features_train = inc_model.predict(train_generator, 2000)
np.save(open('bn_features_train.npy', 'wb'), bottleneck_features_train)

bottleneck_features_validation = inc_model.predict(validation_generator, 2000)
np.save(open('bn_features_validation.npy', 'wb'), bottleneck_features_validation)

In [ ]:
train_data = np.load(open('bn_features_train.npy', 'rb'))
train_labels = np.array([0] * 1000 + [1] * 1000) 

validation_data = np.load(open('bn_features_validation.npy', 'rb'))
validation_labels = np.array([0] * 1000 + [1] * 1000)

In [ ]:
fc_model = Sequential()
fc_model.add(Flatten(input_shape=train_data.shape[1:]))
fc_model.add(Dense(64, activation='relu', name='dense_one'))
fc_model.add(Dropout(0.5, name='dropout_one'))
fc_model.add(Dense(64, activation='relu', name='dense_two'))
fc_model.add(Dropout(0.5, name='dropout_two'))
fc_model.add(Dense(1, activation='sigmoid', name='output'))

fc_model.compile(optimizer='rmsprop', 
              loss='binary_crossentropy', 
              metrics=['accuracy'])

In [ ]:
#validation_labels = validation_labels.reshape(1, -1)
#train_labels = train_labels.reshape(1, -1)
print(validation_data.shape)
print(validation_labels.shape)
print(train_data.shape)
print(train_labels.shape)

(2000, 2, 2, 2048)
(2000,)
(2000, 2, 2, 2048)
(2000,)


In [ ]:
callback = EarlyStopping(monitor='val_loss', patience=3)
callbacks=[callback]

In [ ]:
fc_model.fit(train_data, train_labels, epochs=50, batch_size=32, validation_data=(validation_data, validation_labels), callbacks=[callback])
fc_model.save_weights('fc_inception_cats_dogs_250.hdf5') # сохраняем веса

Epoch 1/50
63/63 [==============================] - 0s 8ms/step - loss: 0.0024 - accuracy: 0.9995 - val_loss: 1.7948 - val_accuracy: 0.9455
Epoch 2/50
63/63 [==============================] - 1s 9ms/step - loss: 0.0043 - accuracy: 0.9975 - val_loss: 1.7436 - val_accuracy: 0.9515
Epoch 3/50
63/63 [==============================] - 1s 12ms/step - loss: 0.0058 - accuracy: 0.9985 - val_loss: 2.0246 - val_accuracy: 0.9400
Epoch 4/50
63/63 [==============================] - 0s 7ms/step - loss: 0.0084 - accuracy: 0.9990 - val_loss: 1.6936 - val_accuracy: 0.9450
Epoch 5/50
63/63 [==============================] - 1s 9ms/step - loss: 0.0017 - accuracy: 0.9990 - val_loss: 1.7769 - val_accuracy: 0.9455
Epoch 6/50
63/63 [==============================] - 0s 7ms/step - loss: 0.0032 - accuracy: 0.9975 - val_loss: 1.7802 - val_accuracy: 0.9470
Epoch 7/50
63/63 [==============================] - 0s 7ms/step - loss: 8.5275e-04 - accuracy: 1.0000 - val_loss: 1.5172 - val_accuracy: 0.9545
Epoch 8/50
63/6

In [ ]:
fc_model.evaluate(validation_data, validation_labels)

63/63 [==============================] - 0s 3ms/step - loss: 1.5300 - accuracy: 0.9490


[1.5300487279891968, 0.9490000009536743]

In [ ]:
weights_filename='fc_inception_cats_dogs_250.hdf5'
x = Flatten()(inc_model.output)
x = Dense(64, activation='relu', name='dense_one')(x)
x = Dropout(0.5, name='dropout_one')(x)
x = Dense(64, activation='relu', name='dense_two')(x)
x = Dropout(0.5, name='dropout_two')(x)
top_model=Dense(1, activation='sigmoid', name='output')(x)
model = Model(inputs=inc_model.input, outputs=top_model)

In [ ]:
weights_filename='fc_inception_cats_dogs_250.hdf5'
model.load_weights(weights_filename, by_name=True)

In [ ]:
for layer in inc_model.layers[:205]:
    layer.trainable = False

In [ ]:
model.compile(loss='binary_crossentropy',
              optimizer=SGD(learning_rate=1e-4, momentum=0.9),
                #optimizer='rmsprop',
              metrics=['accuracy'])

In [ ]:
filepath="weights-improvement-{epoch:02d}-{val_accuracy:.2f}.hdf5"
checkpoint = ModelCheckpoint(filepath,  monitor='val_accuracy', verbose=2, save_best_only=True, mode='max')
early_stop = EarlyStopping(monitor='val_loss',patience=7, verbose=1, mode='auto')
callbacks_list = [checkpoint, early_stop]

In [ ]:
train_datagen = ImageDataGenerator(
        rescale=1./255)#,
        #shear_range=0.2,
        #zoom_range=0.2,
        #horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        '/content/sample_data/data/train/',
        target_size=(116, 116),
        batch_size=32,
        class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
        '/content/sample_data/data/validation/',
        target_size=(116, 116),
        batch_size=32,
        class_mode='binary')


pred_generator=test_datagen.flow_from_directory('/content/sample_data/data/validation/',
                                                     target_size=(116,116),
                                                     batch_size=100,
                                                     class_mode='binary')

Found 2000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.


In [ ]:
steps_per_epoch = 2000 // 32

validation_steps = 2000 // 32

In [ ]:
model.fit(
        train_generator,
        steps_per_epoch=steps_per_epoch,
        epochs=200,
        validation_data=validation_generator,
        validation_steps=validation_steps,
        #nb_val_samples=2000,
        callbacks=callbacks_list)

Epoch 1/200
62/62 [==============================] - ETA: 0s - loss: 0.3577 - accuracy: 0.9685
Epoch 1: val_accuracy improved from -inf to 0.89718, saving model to weights-improvement-01-0.90.hdf5
62/62 [==============================] - 12s 116ms/step - loss: 0.3577 - accuracy: 0.9685 - val_loss: 1.4060 - val_accuracy: 0.8972
Epoch 2/200
62/62 [==============================] - ETA: 0s - loss: 0.0299 - accuracy: 0.9939
Epoch 2: val_accuracy improved from 0.89718 to 0.91331, saving model to weights-improvement-02-0.91.hdf5
62/62 [==============================] - 6s 96ms/step - loss: 0.0299 - accuracy: 0.9939 - val_loss: 0.8801 - val_accuracy: 0.9133
Epoch 3/200
62/62 [==============================] - ETA: 0s - loss: 0.0123 - accuracy: 0.9980
Epoch 3: val_accuracy improved from 0.91331 to 0.94153, saving model to weights-improvement-03-0.94.hdf5
62/62 [==============================] - 6s 96ms/step - loss: 0.0123 - accuracy: 0.9980 - val_loss: 0.4807 - val_accuracy: 0.9415
Epoch 4/200

In [ ]:
model.evaluate_generator(pred_generator, val_samples=100)

TypeError: ignored

In [ ]:
mgs,labels=pred_generator.next()
array_imgs=np.transpose(np.asarray([img_to_array(img) for img in imgs]),(0,2,1,3))
predictions=model.predict(imgs)
rounded_pred=np.asarray([round(i) for i in predictions])

In [ ]:
wrong=[im for im in zip(array_imgs, rounded_pred, labels, predictions) if im[1]!=im[2]]

plt.figure(figsize=(12,12))
for ind, val in enumerate(wrong[:100]):
    plt.subplots_adjust(left=0, right=1, bottom=0, top=1, wspace = 0.2, hspace = 0.2)
    plt.subplot(5,5,ind+1)
    im=val[0]
    plt.axis('off')
    plt.text(120, 0, round(val[3], 2), fontsize=11, color='red')
    plt.text(0, 0, val[2], fontsize=11, color='blue')
    plt.imshow(np.transpose(im,(2,1,0)))

In [ ]:
right=[im for im in zip(array_imgs, rounded_pred, labels, predictions) if im[1]==im[2]]

plt.figure(figsize=(12,12))
for ind, val in enumerate(right[:20]):
    plt.subplots_adjust(left=0, right=1, bottom=0, top=1, wspace = 0.2, hspace = 0.2)
    plt.subplot(5,5,ind+1)
    im=val[0]
    plt.axis('off')
    plt.text(120, 0, round(val[3], 2), fontsize=11, color='red')
    plt.text(0, 0, val[2], fontsize=11, color='blue')
    plt.imshow(np.transpose(im,(2,1,0)))